# Learning to detect unseen object classes by Between-class Attribute Transfer 

Implementaton with features extracted from a VGG19 convnet.

# 1. Loading data

In [1]:
import numpy as np
import pandas as pd
import operator
import pickle as cPickle
import bz2
from scipy.sparse import csr_matrix
from sklearn.svm import SVC

PATH = "./Dataset/Animals_with_Attributes/"
DATA_PATH = "/media/corbi/Data/Dataset/AwA/"

#### Classes index

In [2]:
animal_dict = {}
with open("classes.txt") as f:
    for line in f:
        (key, val) = line.split()
        animal_dict[val] = int(key)
        
inv_animal_index = sorted(animal_dict.items(), key=operator.itemgetter(1))

#### Train classes

In [3]:
trainclasses = []
with open('trainclasses.txt') as infile:
    for line in infile:
        trainclasses.append(line[:-1])
train_animal_index = [animal_dict[animal]-1 for animal in trainclasses]

print (trainclasses)

['antelope', 'grizzly+bear', 'killer+whale', 'beaver', 'dalmatian', 'horse', 'german+shepherd', 'blue+whale', 'siamese+cat', 'skunk', 'mole', 'tiger', 'moose', 'spider+monkey', 'elephant', 'gorilla', 'ox', 'fox', 'sheep', 'hamster', 'squirrel', 'rhinoceros', 'rabbit', 'bat', 'giraffe', 'wolf', 'chihuahua', 'weasel', 'otter', 'buffalo', 'zebra', 'deer', 'bobcat', 'lion', 'mouse', 'polar+bear', 'collie', 'walrus', 'cow', 'dolphin']


#### Test classes

In [4]:
testclasses = []
with open('testclasses.txt') as infile:
    for line in infile:
        testclasses.append(line[:-1])
test_animal_index = [animal_dict[animal]-1 for animal in testclasses]

print (testclasses)

['chimpanzee', 'giant+panda', 'leopard', 'persian+cat', 'pig', 'hippopotamus', 'humpback+whale', 'raccoon', 'rat', 'seal']


### Creating data

In [5]:
def bzUnpickle(filename):
    return cPickle.load(bz2.BZ2File(filename))

#### Loading class-attributes relation ship 

In [6]:
classAttributes = np.loadtxt("predicate-matrix-binary.txt", comments="#", unpack=False)
NB_ATTRIBUTES = classAttributes.shape[1]

train_attributes = classAttributes[train_animal_index]
test_attributes = classAttributes[test_animal_index]
print ('Number of attributes:', NB_ATTRIBUTES)

Number of attributes: 85


#### Loading data from path and index

In [7]:
def create_data(X_path, sample_index, attributes):
  
    X = bzUnpickle('CreatedData/train_featuresVGG19.pic.bz2')
    
    nb_animal_samples = [item[1] for item in sample_index]
    for i,nb_samples in enumerate(nb_animal_samples):
        if i==0:
            y = np.array([attributes[i,:]]*nb_samples)
        else:
            y = np.concatenate((y,np.array([attributes[i,:]]*nb_samples)), axis=0)
    
    return X,y

In [8]:
train_index = bzUnpickle('CreatedData/train_features_index.txt')
test_index = bzUnpickle('CreatedData/test_features_index.txt')

In [9]:
X_train, y_train = create_data('CreatedData/train_featuresVGG19.pic.bz2',train_index, train_attributes)

In [31]:
X_test, y_test = create_data('CreatedData/test_featuresVGG19.pic.bz2',test_index, test_attributes)

In [45]:
X_train

<24295x4096 sparse matrix of type '<class 'numpy.float64'>'
	with 24942942 stored elements in Compressed Sparse Row format>

In [46]:
y_train.shape

(24295, 85)

# 2. Run SVM for each attribute

In [ ]:
SVMs = []
for i in range(5):
    print ('Fitting classifier for attribute %d/%d' % (i+1,5))
    clf = SVC()
    clf.fit(X_train, y_train[:,i]) 
    SVMs.append(clf)

Fitting classifier for attribute 1/5


In [1]:
for i in range(5):
    print ('Predicting for attribute %d/%d' % (i+1,NB_ATTRIBUTES))
    y_pred = SVMs[i].predict(X_test)

NameError: name 'NB_ATTRIBUTES' is not defined

# 3. Neural network

In [41]:
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras import optimizers

N = X_train.shape[1]

model = Sequential()
model.add(Dense(NB_ATTRIBUTES, input_dim=N))
model.add(Activation("relu"))
model.compile(optimizer="adam", loss=binary_crossentropy) 

In [49]:
history = model.fit(X_train[:1000,:].toarray(), y_train[:1000,:], batch_size=10, nb_epoch=10)

Epoch 1/10
1000/1000 [==============================] - 0s - loss: 0.5892     